# SOCAM QuickStart Tutorial
Please do make sure that you have python version 3.10 before running the cells below

This Notebook is a short tutorial on using SOCAM model variation 3 (probability propagation during training and inference) to predict cell types on given query samples. Please read the instructions and run each cell in the presented order. Once you have finished going through the tutorial, please feel free to go back and modify it as needed for annotating your own datasets.

### Installing cellarium-ml client library (SOCAM branch)

As a first step, we need to install the cellarium-ml SOCAM branch with all ``cellarium-ml`` related dependencies. To this end, run the next cell.

In [17]:
!pip install git+https://github.com/cellarium-ai/cellarium-ml.git@SOCAM

  Cloning https://github.com/cellarium-ai/cellarium-ml.git (to revision SOCAM) to /var/tmp/pip-req-build-ywbe6udf
  Running command git clone --filter=blob:none --quiet https://github.com/cellarium-ai/cellarium-ml.git /var/tmp/pip-req-build-ywbe6udf
  Running command git checkout -b SOCAM --track origin/SOCAM
  Switched to a new branch 'SOCAM'
  Branch 'SOCAM' set up to track remote branch 'SOCAM' from 'origin'.
  Resolved https://github.com/cellarium-ai/cellarium-ml.git to commit eb5ab2149bf53e06b95477d03fe55b3e166bd449
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached anndata-0.11.4-py3-none-any.whl.metadata (9.3 kB)
  Using cached boltons-25.0.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached braceexpand-0.1.7-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached crick-0.0.8-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.1 kB)
  Using cached jsonargparse-4.27

In [18]:
!pip install owlready2

  Using cached owlready2-0.48-cp310-cp310-linux_x86_64.whl


# Load sample validation anndata object

In [20]:
# Helpful functions to import fom the cellarium-ml package
from cellarium.ml.data.fileio import read_h5ad_gcs
from cellarium.ml.data.fileio import read_pkl_from_gcs
import pandas as pd
from cellarium.ml import CellariumModule, CellariumAnnDataDataModule
from cellarium.ml.utilities.data import AnnDataField, densify, filter_categories

In [21]:
adata = read_h5ad_gcs('gs://cellarium-file-system-cas-archive/curriculum/lrexp_human_validation_split_20241126/extract_files/extract_10.h5ad')


/opt/conda/envs/socam_tutorial/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/envs/socam_tutorial/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/envs/socam_tutorial/lib/python3.10/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificati

# Load the trained model from checkpoint

In [22]:
# load SOCAM model from checkpoint
socam_module = CellariumModule.load_from_checkpoint("gs://cellarium-file-system-cas-archive/curriculum/lrexp_human_training_split_20241106/models/Model_variation_3/lightning_logs/version_1/checkpoints/last.ckpt")


# Define the Data Module

In [23]:
datamodule = CellariumAnnDataDataModule(
    dadc=adata,
    batch_keys={
        "x_ng":AnnDataField(attr='X', convert_fn=densify),
        "var_names_g":AnnDataField(attr="var_names"),
        "total_mrna_umis_n":AnnDataField(attr='obs', key='total_mrna_umis'),
        "y_n":AnnDataField(attr='obs', key='cell_type_ontology_term_id',convert_fn=filter_categories),
        "obs_names_n": AnnDataField(attr="obs_names", convert_fn=filter_categories)
    },
    batch_size = len(adata),
    iteration_strategy= "cache_efficient",
    shuffle= False,
    seed= 0,
    drop_last_indices= False,
    num_workers= 4,
    drop_incomplete_batch= False
)

datamodule.setup(stage='predict')



# Define Trainer module

In [24]:
import lightning.pytorch as pl
trainer = pl.Trainer(
    accelerator="cpu",
    devices=1,
    num_nodes=1,
    precision="32-true",
    inference_mode= True,
)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/envs/socam_tutorial/lib/python3.10/site-packages/lightning/pytorch/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


# Define important initialization arguments for CustomLogisticRegression

In [25]:
socam_module.model.actual_categories = 2613 # total cell type categories to predict
socam_module.model.valid_mask = read_pkl_from_gcs('gs://cellarium-file-system-cas-archive/curriculum/lrexp_human_validation_split_20241126/shared_meta/sublist_indices.pkl')
socam_module.model.target_row_descendent_col_torch_tensor = read_pkl_from_gcs('gs://cellarium-file-system-cas-archive/curriculum/lrexp_human_training_split_20241106/models/shared_metadata/target_row_descendent_col_torch_tensor_lrexp_human.pkl')
socam_module.model.probability_propagation_flag = True #used for both training and validation
socam_module.model.y_categories_path = 'gs://cellarium-file-system-cas-archive/curriculum/lrexp_human_training_split_20241106/models/shared_metadata/final_filtered_sorted_unique_cells_lrexp_human.pkl',
    

/opt/conda/envs/socam_tutorial/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/envs/socam_tutorial/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/envs/socam_tutorial/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your appl

# Run prediction with the loaded modules

In [26]:
output = trainer.predict(
    model=socam_module,
    datamodule=datamodule
)


/opt/conda/envs/socam_tutorial/lib/python3.10/site-packages/cellarium/ml/utilities/distributed.py:52: UserWarning: Distributed package is available but the default process group has not been initialized. Falling back to ``rank=0`` and ``num_replicas=1``.
  warnings.warn(
/opt/conda/envs/socam_tutorial/lib/python3.10/site-packages/lightning/pytorch/utilities/data.py:123: Your `IterableDataset` has `__len__` defined. In combination with multi-process data loading (when num_workers > 1), `__len__` could be inaccurate if each worker is not configured independently to avoid having duplicate data.


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  4.56it/s]


In [27]:
# lets take a look at the probability score output shape 
# (expect it to be number of samples, number of predicted clases)
output[0]['cell_type_probs_nc'].shape

torch.Size([815, 2613])

In [28]:
# Lets save the output probability score array to the adata object
adata.obsm['socam_cl_scores'] = output[0]['cell_type_probs_nc'].detach().numpy()

# Single Label Prediction

In [29]:
from cellarium.ml.single_label_prediction import cell_ontology_cache
cl=cell_ontology_cache.CellOntologyCache()
min_acceptable_score = 0.2
top_k=2
cl_labels = read_pkl_from_gcs('gs://cellarium-file-system-cas-archive/curriculum/lrexp_human_training_split_20241106/models/shared_metadata/ontology_term_id_to_cell_type_np_array_lrexp_human.pkl')
cl_names = read_pkl_from_gcs('gs://cellarium-file-system-cas-archive/curriculum/lrexp_human_training_split_20241106/models/shared_metadata/final_filtered_sorted_unique_cells_lrexp_human.pkl')
cl_names=[s.replace(':', '_') for s in cl_names]
cl_names_to_labels_map = dict(zip(cl_names, cl_labels))
cl_names_to_labels_map['CL_0000000'] = 'cell'

/opt/conda/envs/socam_tutorial/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/envs/socam_tutorial/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/envs/socam_tutorial/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your appl

In [30]:
# Loading helper function
from cellarium.ml.single_label_prediction.ontology_aware import compute_most_granular_top_k_calls_single


In [31]:
# lets store the output in adata.granular_results
adata.granular_results = pd.DataFrame()

In [32]:
compute_most_granular_top_k_calls_single(
    adata=adata,
    cl=cell_ontology_cache.CellOntologyCache(),
    min_acceptable_score=0.2,
    cl_names=cl_names,
    cl_names_to_labels_map=cl_names_to_labels_map,
    top_k=top_k
)

# Lets take a quick look at the results

In [33]:
adata.granular_results

,cas_cell_type_score_1,cas_cell_type_name_1,cas_cell_type_label_1,cas_cell_type_score_2,cas_cell_type_name_2,cas_cell_type_label_2
0,0.399896,CL_0000904,"central memory CD4-positive, alpha-beta T cell",0.403772,CL_0001204,"CD4-positive, alpha-beta memory T cell, CD45RO..."
1,0.290486,CL_0002057,"CD14-positive, CD16-negative classical monocyte",0.702609,CL_0002396,"CD14-low, CD16-positive monocyte"
2,0.214840,CL_0000904,"central memory CD4-positive, alpha-beta T cell",0.215455,CL_0001204,"CD4-positive, alpha-beta memory T cell, CD45RO..."
3,0.817067,CL_0000895,"naive thymus-derived CD4-positive, alpha-beta ...",0.961544,CL_0000624,"CD4-positive, alpha-beta T cell"
4,0.978470,CL_0002057,"CD14-positive, CD16-negative classical monocyte",0.978700,CL_0000860,classical monocyte
...,...,...,...,...,...,...
810,0.738824,CL_0000904,"central memory CD4-positive, alpha-beta T cell",0.746131,CL_0001204,"CD4-positive, alpha-beta memory T cell, CD45RO..."
811,0.993852,CL_0002057,"CD14-positive, CD16-negative classical monocyte",0.993914,CL_0000860,classical monocyte
812,0.659642,CL_0002057,"CD14-positive, CD16-negative classical monocyte",0.335533,CL_0002396,"CD14-low, CD16-positive monocyte"
813,0.930776,CL_0000904,"central memory CD4-positive, alpha-beta T cell",0.935689,CL_0001204,"CD4-positive, alpha-beta memory T cell, CD45RO..."
